# VOCS v1 data structure 

In [1]:
from xopt.vocs import VOCS, ObjectiveEnum, ConstraintEnum

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data, prefix='constraint_')
 |  
 |  convert_dataframe_to_inputs(self, data: pandas.core.frame.DataFrame) -> pandas.core.frame.DataFrame
 |      Extracts only inputs from a dataframe.
 |      This will add constants.
 |  
 |  convert_numpy_to_inputs(self, inputs: numpy.ndarra

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == ObjectiveEnum.MAXIMIZE

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

In [10]:
vocs.schema()

{'title': 'VOCS',
 'description': 'Variables, Objectives, Constraints, and other Settings (VOCS) data structure\nto describe optimization problems.',
 'type': 'object',
 'properties': {'variables': {'title': 'Variables',
   'default': {},
   'type': 'object',
   'additionalProperties': {'type': 'array',
    'items': {'type': 'number'},
    'minItems': 2,
    'maxItems': 2}},
  'constraints': {'title': 'Constraints',
   'default': {},
   'type': 'object',
   'additionalProperties': {'type': 'array',
    'items': {'anyOf': [{'type': 'number'},
      {'$ref': '#/definitions/ConstraintEnum'}]},
    'minItems': 2,
    'maxItems': 2}},
  'objectives': {'default': {},
   'type': 'object',
   'additionalProperties': {'$ref': '#/definitions/ObjectiveEnum'}},
  'constants': {'title': 'Constants', 'default': {}, 'type': 'object'},
  'linked_variables': {'title': 'Linked Variables',
   'default': {},
   'type': 'object',
   'additionalProperties': {'type': 'string'}}},
 'additionalProperties': Fal

# Objective Evaluation

In [11]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,584.485344,0.179413,1234,584.664757,584.305930,1169.329514,1168.611861
6,488.229632,-0.039953,1234,488.189679,488.269585,976.379357,976.539171
7,15.183497,-0.168647,1234,15.014850,15.352144,30.029700,30.704289
8,654.054492,0.311430,1234,654.365922,653.743063,1308.731844,1307.486125
9,473.446522,0.670863,1234,474.117385,472.775659,948.234771,945.551318
10,997.594137,0.987013,1234,998.581150,996.607124,1997.162300,1993.214247
11,702.018284,-0.638901,1234,701.379384,702.657185,1402.758767,1405.314370
12,469.606236,-0.460899,1234,469.145337,470.067135,938.290675,940.134269
13,378.981615,-0.655947,1234,378.325668,379.637562,756.651337,759.275125
14,356.755563,-0.677893,1234,356.077670,357.433456,712.155341,714.866912


In [12]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [13]:
?form_objective_data

Signature: form_objective_data(objectives: Dict, data, prefix='objective_')
Docstring:
Use objective dict and data (dataframe) to generate objective data (dataframe)

Weights are applied to convert all objectives into mimimization form.

Returns a dataframe with the objective data intented to be minimized.

Missing or nan values will be filled with: np.inf 
File:      ~/Code/GitHub/Xopt/xopt/vocs.py
Type:      function


In [14]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-584.664757,584.305930
6,-488.189679,488.269585
7,-15.014850,15.352144
8,-654.365922,653.743063
9,-474.117385,472.775659
10,-998.581150,996.607124
11,-701.379384,702.657185
12,-469.145337,470.067135
13,-378.325668,379.637562
14,-356.077670,357.433456


In [15]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-584.664757,584.305930
6,-488.189679,488.269585
7,-15.014850,15.352144
8,-654.365922,653.743063
9,-474.117385,472.775659
10,-998.581150,996.607124
11,-701.379384,702.657185
12,-469.145337,470.067135
13,-378.325668,379.637562
14,-356.077670,357.433456


In [16]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-584.66475694,  584.30593036],
       [-488.18967865,  488.26958528],
       [ -15.01485007,   15.35214431],
       [-654.36592197,  653.74306251],
       [-474.11738539,  472.7756588 ],
       [-998.58115017,  996.60712356],
       [-701.37938363,  702.65718478],
       [-469.14533726,  470.06713461],
       [-378.32566835,  379.63756234],
       [-356.07767048,  357.4334559 ]])

In [17]:
?form_constraint_data

Signature:
form_constraint_data(
    constraints: Dict,
    data: pandas.core.frame.DataFrame,
    prefix='constraint_',
)
Docstring:
Use constraint dict and data (dataframe) to generate constraint data (dataframe)
A constraint is satisfied if the evaluation is < 0.

Args:
    constraints: Dictonary of constraints
    data: Dataframe with the data to be evaluated
    prefix: Prefix to use for the transformed data in the dataframe

Returns a dataframe with the constraint data.

Missing or nan values will be filled with: np.inf 
File:      ~/Code/GitHub/Xopt/xopt/vocs.py
Type:      function


In [18]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,1167.329514,-1168.611861
6,974.379357,-976.539171
7,28.029700,-30.704289
8,1306.731844,-1307.486125
9,946.234771,-945.551318
10,1995.162300,-1993.214247
11,1400.758767,-1405.314370
12,936.290675,-940.134269
13,754.651337,-759.275125
14,710.155341,-714.866912


In [19]:
?form_feasibility_data

Signature: form_feasibility_data(constraints: Dict, data, prefix='feasible_')
Docstring:
Use constraint dict and data to identify feasible points in the the dataset.

Returns a dataframe with the feasibility data.
File:      ~/Code/GitHub/Xopt/xopt/vocs.py
Type:      function


In [20]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [21]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [22]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [23]:
data['a']  = np.nan

In [24]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [25]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [26]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
